# Setup

In [ ]:
pip install pyyaml h5py  

In [ ]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.5.0


# Get an example dataset

In [ ]:
# 데이터 불러오기
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

11493376/11490434 [==============================] - 0s 0us/step


# Define a model

In [ ]:
# 간단한 모델 생성
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.metrics.SparseCategoricalAccuracy()])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


# Save checkpoints during training

In [ ]:
# 저장 경로
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 모델 가중치 기록하는 callback 생성
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# callback과 함께 모델 학습
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback]) 

Epoch 1/10
32/32 [==============================] - 1s 12ms/step - loss: 1.1721 - sparse_categorical_accuracy: 0.6630 - val_loss: 0.6809 - val_sparse_categorical_accuracy: 0.8020

Epoch 00001: saving model to training_1/cp.ckpt
Epoch 2/10
32/32 [==============================] - 0s 7ms/step - loss: 0.4255 - sparse_categorical_accuracy: 0.8860 - val_loss: 0.5179 - val_sparse_categorical_accuracy: 0.8470

Epoch 00002: saving model to training_1/cp.ckpt
Epoch 3/10
32/32 [==============================] - 0s 7ms/step - loss: 0.2906 - sparse_categorical_accuracy: 0.9200 - val_loss: 0.4574 - val_sparse_categorical_accuracy: 0.8560

Epoch 00003: saving model to training_1/cp.ckpt
Epoch 4/10
32/32 [==============================] - 0s 7ms/step - loss: 0.2093 - sparse_categorical_accuracy: 0.9420 - val_loss: 0.4389 - val_sparse_categorical_accuracy: 0.8540

Epoch 00004: saving model to training_1/cp.ckpt
Epoch 5/10
32/32 [==============================] - 0s 7ms/step - loss: 0.1597 - sparse_cat

In [ ]:
os.listdir(checkpoint_dir)

['cp.ckpt.index', 'checkpoint', 'cp.ckpt.data-00000-of-00001']

In [ ]:
# 모델 생성
model = create_model()

# 모델 평가
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 2.3386 - sparse_categorical_accuracy: 0.1240
Untrained model, accuracy: 12.40%


In [ ]:
# 가중치 불러오기(모델의 구조가 동일하기에 가능)
model.load_weights(checkpoint_path)

# 모델 평가
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4078 - sparse_categorical_accuracy: 0.8730
Restored model, accuracy: 87.30%


# Checkpoint callback options

In [ ]:
# 에폭을 포함한 파일명
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

# 5에폭마다 저장할 callback 생성
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=5*batch_size)

# 모델 생성
model = create_model()

# 0에폭 가중치 저장
model.save_weights(checkpoint_path.format(epoch=0))

# callback과 함께 모델 학습
model.fit(train_images, 
          train_labels,
          epochs=50, 
          batch_size=batch_size, 
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [ ]:
# 5에폭마다 저장된 checkpoint들 확인
os.listdir(checkpoint_dir)

['cp-0010.ckpt.index',
 'cp-0035.ckpt.index',
 'cp-0025.ckpt.index',
 'cp-0025.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.data-00000-of-00001',
 'cp-0015.ckpt.index',
 'cp-0040.ckpt.index',
 'cp-0030.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.data-00000-of-00001',
 'cp-0045.ckpt.data-00000-of-00001',
 'checkpoint',
 'cp-0050.ckpt.index',
 'cp-0005.ckpt.data-00000-of-00001',
 'cp-0030.ckpt.index',
 'cp-0010.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.index',
 'cp-0015.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.index',
 'cp-0040.ckpt.data-00000-of-00001',
 'cp-0045.ckpt.index',
 'cp-0005.ckpt.index',
 'cp-0050.ckpt.data-00000-of-00001']

# Save the entire model

In [ ]:
# 모델 생성 및 학습
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# 모델 저장
!mkdir -p saved_model
model.save('saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 0s 5ms/step - loss: 1.1202 - sparse_categorical_accuracy: 0.6740
Epoch 2/5
32/32 [==============================] - 0s 5ms/step - loss: 0.4099 - sparse_categorical_accuracy: 0.8830
Epoch 3/5
32/32 [==============================] - 0s 5ms/step - loss: 0.2753 - sparse_categorical_accuracy: 0.9370
Epoch 4/5
32/32 [==============================] - 0s 5ms/step - loss: 0.2023 - sparse_categorical_accuracy: 0.9490
Epoch 5/5
32/32 [==============================] - 0s 5ms/step - loss: 0.1571 - sparse_categorical_accuracy: 0.9700
INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [ ]:
# 모델 불러오기
new_model = tf.keras.models.load_model('saved_model/my_model')

new_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# 모델 평가
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4387 - sparse_categorical_accuracy: 0.8590
Restored model, accuracy: 85.90%
(1000, 10)


## HDF5 format

In [ ]:
# 모델 생성 및 학습
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# HDF5 확장자로 저장
model.save('my_model.h5')

Epoch 1/5
32/32 [==============================] - 0s 5ms/step - loss: 1.1787 - sparse_categorical_accuracy: 0.6540
Epoch 2/5
32/32 [==============================] - 0s 4ms/step - loss: 0.4113 - sparse_categorical_accuracy: 0.8880
Epoch 3/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2844 - sparse_categorical_accuracy: 0.9300
Epoch 4/5
32/32 [==============================] - 0s 5ms/step - loss: 0.2045 - sparse_categorical_accuracy: 0.9460
Epoch 5/5
32/32 [==============================] - 0s 4ms/step - loss: 0.1662 - sparse_categorical_accuracy: 0.9610


In [ ]:
# 모델 불러오기(weights, optimizer 포함)
new_model = tf.keras.models.load_model('my_model.h5')

new_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# 모델 평가
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

32/32 - 0s - loss: 0.4168 - sparse_categorical_accuracy: 0.8550
Restored model, accuracy: 85.50%
